# Decision trees tryouts on SPR data, inspired by Kaggle Forum "When less is more"

Load training and validation data as 
    month : [ Features | Targets| Difference | Last Choice Targets  ]
    

In [ ]:
# https://ipython.org/ipython-doc/3/config/extensions/autoreload.html
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import numpy as np
import pandas as pd

import logging
logging.getLogger().handlers = []
logging.basicConfig(level=logging.INFO)

import matplotlib.pylab as plt
%matplotlib inline

In [ ]:
import sys
sys.path.append("../common")

from dataset import load_trainval, LC_TARGET_LABELS, TARGET_LABELS_FRQ, TARGET_LABELS_DIFF
from utils import to_yearmonth, TARGET_LABELS, TARGET_LABELS2
from utils import target_str_to_labels, decimal_to_dummies, targets_str_to_indices, targets_dec_to_indices

In [4]:
features = [
    u'ind_empleado', u'pais_residencia',
    u'sexo', u'age', u'ind_nuevo', u'antiguedad', u'indrel',
    u'ult_fec_cli_1t', u'indrel_1mes', u'tiprel_1mes', u'indresi',
    u'indext', u'conyuemp', u'canal_entrada', u'indfall', u'nomprov',
    u'ind_actividad_cliente', u'renta', u'segmento'    
]

In [5]:
# # train_yearmonths_list = [201504, 201505, 201604]
# train_yearmonths_list = [201505, 201602, 201605]
# # train_yearmonths_list = [201505]
# #val_yearmonth = [201605]
# train_nb_clients = 150000
# # train_nb_clients = 1500
# #train_df, val_df = load_trainval(train_yearmonths_list, val_yearmonth, train_nb_clients, val_nb_clients=1500)
# train_df = load_trainval(train_yearmonths_list, train_nb_clients=train_nb_clients)
filename = "trainval_201505+201602+201605__150000.csv"
train_df = pd.read_csv('../data/generated/' + filename)

Display loaded data

In [6]:
train_df[['fecha_dato', 'ncodpers'] + TARGET_LABELS_FRQ.tolist()].head(10)

,fecha_dato,ncodpers,ind_ahor_fin_ult1_frq,ind_aval_fin_ult1_frq,ind_cco_fin_ult1_frq,ind_cder_fin_ult1_frq,ind_cno_fin_ult1_frq,ind_ctju_fin_ult1_frq,ind_ctma_fin_ult1_frq,ind_ctop_fin_ult1_frq,...,ind_hip_fin_ult1_frq,ind_plan_fin_ult1_frq,ind_pres_fin_ult1_frq,ind_reca_fin_ult1_frq,ind_tjcr_fin_ult1_frq,ind_valo_fin_ult1_frq,ind_viv_fin_ult1_frq,ind_nomina_ult1_frq,ind_nom_pens_ult1_frq,ind_recibo_ult1_frq
0,2015-04-28,15889,0.999878,0.999984,0.771587,0.999491,0.897256,0.988464,0.990227,0.835914,...,0.992618,0.987667,0.996794,0.931887,0.94339,0.033643,0.995141,0.932789,0.927527,0.839655
1,2015-05-28,15889,0.999878,0.999984,0.771587,0.999491,0.897256,0.988464,0.990227,0.835914,...,0.992618,0.987667,0.996794,0.931887,0.05661,0.033643,0.995141,0.932789,0.927527,0.839655
2,2016-01-28,15889,0.999878,0.999984,0.771587,0.999491,0.897256,0.988464,0.990227,0.835914,...,0.992618,0.987667,0.996794,0.931887,0.05661,0.033643,0.995141,0.932789,0.927527,0.839655
3,2016-02-28,15889,0.999878,0.999984,0.771587,0.999491,0.897256,0.988464,0.990227,0.835914,...,0.992618,0.987667,0.996794,0.931887,0.94339,0.033643,0.995141,0.932789,0.927527,0.839655
4,2016-04-28,15889,0.999878,0.999984,0.771587,0.999491,0.897256,0.988464,0.990227,0.835914,...,0.992618,0.987667,0.996794,0.931887,0.94339,0.033643,0.995141,0.932789,0.927527,0.839655
5,2016-05-28,15889,0.999878,0.999984,0.771587,0.999491,0.897256,0.988464,0.990227,0.835914,...,0.992618,0.987667,0.996794,0.931887,0.05661,0.033643,0.995141,0.932789,0.927527,0.839655
6,2015-04-28,15893,0.999878,0.999984,0.228413,0.999491,0.897256,0.988464,0.990227,0.835914,...,0.992618,0.987667,0.996794,0.931887,0.94339,0.033643,0.995141,0.932789,0.927527,0.839655
7,2015-05-28,15893,0.999878,0.999984,0.228413,0.999491,0.897256,0.988464,0.990227,0.835914,...,0.992618,0.987667,0.996794,0.931887,0.94339,0.033643,0.995141,0.932789,0.927527,0.839655
8,2016-01-28,15893,0.999878,0.999984,0.228413,0.999491,0.897256,0.988464,0.990227,0.835914,...,0.992618,0.987667,0.996794,0.931887,0.94339,0.033643,0.995141,0.932789,0.927527,0.839655
9,2016-02-28,15893,0.999878,0.999984,0.228413,0.999491,0.897256,0.988464,0.990227,0.835914,...,0.992618,0.987667,0.996794,0.931887,0.94339,0.033643,0.995141,0.932789,0.927527,0.839655


Useful structures

In [7]:
def get_common_clients(df1, mask1, mask2, df2=None):
    active_clients1 = df1[mask1]['ncodpers'].unique()
    if df2 is not None:
        active_clients2 = df2[mask2]['ncodpers'].unique()
    else:
        active_clients2 = df1[mask2]['ncodpers'].unique()
    active_clients = list(set(active_clients1) & set(active_clients2)) 
    
    if df2 is not None:
        return df1['ncodpers'].isin(active_clients), df2['ncodpers'].isin(active_clients)
    return df1['ncodpers'].isin(active_clients)

In [8]:
months_ym_map = {}
# months = list(set(train_df['fecha_dato'].unique()) | set(val_df['fecha_dato'].unique()))
months = train_df['fecha_dato'].unique()
for m in months:
    months_ym_map[to_yearmonth(m)] = m

        
train_months = train_df['fecha_dato'].unique()
# val_months = val_df['fecha_dato'].unique()
    

### Train a model

In [9]:
from utils import get_added_products, remove_last_choice, apk, map7_score
from visualization import visualize_train_test, visualize_folds, compare_two_datasets, compare_folds, compare_folds2

In [10]:
target_features = ['targets_diff', 'targets_logdiff', 'targets_logcount2_diff', 'targets_logcount2', 'targets_logcount1', 'targets_logDec']
TARGET_LABELS_FRQ_PREV = [c + '_prev' for c in TARGET_LABELS_FRQ]

In [11]:
def get_XY(current_month, df1, next_year_month, df2, months_ym_map):
    month_mask = df1['fecha_dato'] == months_ym_map[current_month]
    next_year_month_mask = df2['fecha_dato'] == months_ym_map[next_year_month]
    next_year_prev_month_mask = df2['fecha_dato'] == months_ym_map[next_year_month - 1]
    
    # get common clients from df1 at this month and df2 at next year month
    common_clients_mask1, common_clients_mask2 = get_common_clients(df1, month_mask, next_year_month_mask, df2)
    common_clients_mask2, common_clients_mask3 = get_common_clients(df2, common_clients_mask2 & next_year_month_mask, next_year_prev_month_mask, df2)
        
    c1 = df1[common_clients_mask1 & month_mask]['ncodpers'].values
    c2 = df2[common_clients_mask2 & next_year_month_mask]['ncodpers'].values
    c3 = df2[common_clients_mask3 & next_year_prev_month_mask]['ncodpers'].values
    assert (c1 == c2).all() and (c2 == c3).all(), "Problem with common clients" 
    
    X = df1[common_clients_mask1 & month_mask][['ncodpers', 'fecha_dato'] + target_features + features + TARGET_LABELS_FRQ.tolist()]            
   
    if TARGET_LABELS[0] in df2.columns and TARGET_LABELS_DIFF[0] in df2.columns and not df2[next_year_month_mask][TARGET_LABELS].isnull().all().all():
        Y = df2[common_clients_mask2 & next_year_month_mask][['ncodpers', 'fecha_dato', 'targets_str', 'lc_targets_str', 'targets_diff'] + TARGET_LABELS + TARGET_LABELS_DIFF.tolist()]    
        assert (X['ncodpers'].values == Y['ncodpers'].values).all(), "There is a problem in alignment"
        Y.index = X.index                
    else:
        Y = None
        
    if TARGET_LABELS_FRQ[0] in df2.columns and not df2[next_year_prev_month_mask][TARGET_LABELS].isnull().all().all():
        # Add TARGET_LABELS_FRQ from previous month to X:
        target_labels_frq = df2[common_clients_mask3 & next_year_prev_month_mask][['ncodpers'] + TARGET_LABELS_FRQ.tolist()]
        assert (X['ncodpers'].values == target_labels_frq['ncodpers'].values).all(), "There is a problem in alignment"
        target_labels_frq = target_labels_frq[TARGET_LABELS_FRQ]
        target_labels_frq.columns = TARGET_LABELS_FRQ_PREV
        target_labels_frq.index = X.index
        X = pd.concat([X, target_labels_frq], axis=1)        

    
    if LC_TARGET_LABELS[0] in df2.columns:
        clients_last_choice = df2[common_clients_mask2 & next_year_month_mask][['ncodpers', 'fecha_dato', 'targets_str'] + LC_TARGET_LABELS.tolist()]
    else:
        clients_last_choice = None
        
    return X, Y, clients_last_choice


In [12]:
current_month = 201505
next_year_month = current_month + 100

df1 = train_df if months_ym_map[current_month] in train_months else val_df
#df1 = train_df
df2 = train_df if months_ym_map[next_year_month] in train_months else val_df
#df2 = train_df

X, Y, clients_last_choice = get_XY(current_month, df1, next_year_month, df2, months_ym_map)

In [13]:
assert (X['ncodpers'].values == Y['ncodpers'].values).all(), "WTF"
assert (X['ncodpers'].values == clients_last_choice['ncodpers'].values).all(), "WTF"

In [14]:
print X.shape
X.head(10)

(149983, 75)


,ncodpers,fecha_dato,targets_diff,targets_logdiff,targets_logcount2_diff,targets_logcount2,targets_logcount1,targets_logDec,ind_empleado,pais_residencia,...,ind_hip_fin_ult1_frq_prev,ind_plan_fin_ult1_frq_prev,ind_pres_fin_ult1_frq_prev,ind_reca_fin_ult1_frq_prev,ind_tjcr_fin_ult1_frq_prev,ind_valo_fin_ult1_frq_prev,ind_viv_fin_ult1_frq_prev,ind_nomina_ult1_frq_prev,ind_nom_pens_ult1_frq_prev,ind_recibo_ult1_frq_prev
1,15889,2015-05-28,32.0,3.496508,-0.00042,0.000047,0.000053,14.571618,0,0,...,0.992618,0.987667,0.996794,0.931887,0.94339,0.033643,0.995141,0.932789,0.927527,0.839655
7,15893,2015-05-28,0.0,0.000000,0.00000,0.001413,0.001453,2.833213,1,0,...,0.992618,0.987667,0.996794,0.931887,0.94339,0.033643,0.995141,0.932789,0.927527,0.839655
13,15895,2015-05-28,0.0,0.000000,0.00000,0.000002,0.000007,14.781716,2,0,...,0.992618,0.012333,0.996794,0.068113,0.05661,0.033643,0.995141,0.932789,0.927527,0.160345
19,15897,2015-05-28,-2.0,-1.098612,0.00000,0.000002,0.000007,14.805207,2,0,...,0.992618,0.012333,0.996794,0.068113,0.05661,0.033643,0.995141,0.932789,0.072473,0.160345
25,15900,2015-05-28,0.0,0.000000,0.00000,0.000261,0.000307,13.287691,3,0,...,0.992618,0.987667,0.996794,0.068113,0.94339,0.966357,0.995141,0.932789,0.927527,0.160345
31,15916,2015-05-28,0.0,0.000000,0.00000,0.000007,0.000007,13.234752,3,0,...,0.992618,0.012333,0.996794,0.068113,0.05661,0.033643,0.995141,0.932789,0.927527,0.160345
37,15919,2015-05-28,0.0,0.000000,0.00000,0.000058,0.000047,14.588785,3,0,...,0.992618,0.987667,0.996794,0.068113,0.94339,0.966357,0.995141,0.932789,0.927527,0.160345
43,15920,2015-05-28,0.0,0.000000,0.00000,0.000052,0.000100,14.803952,0,0,...,0.992618,0.987667,0.996794,0.068113,0.94339,0.033643,0.995141,0.932789,0.927527,0.160345
49,15921,2015-05-28,0.0,0.000000,0.00000,0.000002,0.000007,13.296737,0,0,...,0.992618,0.012333,0.996794,0.931887,0.05661,0.033643,0.995141,0.932789,0.927527,0.160345
55,15922,2015-05-28,0.0,0.000000,0.00000,0.000064,0.000067,14.586867,3,0,...,0.992618,0.987667,0.996794,0.931887,0.94339,0.966357,0.004859,0.932789,0.927527,0.160345


In [15]:
print Y.shape
Y[Y['targets_diff'] > 0][['fecha_dato', 'ncodpers', 'targets_str', 'lc_targets_str'] + TARGET_LABELS_DIFF.tolist() ].head(10)

(149983, 53)


,fecha_dato,ncodpers,targets_str,lc_targets_str,ind_ahor_fin_ult1_diff,ind_aval_fin_ult1_diff,ind_cco_fin_ult1_diff,ind_cder_fin_ult1_diff,ind_cno_fin_ult1_diff,ind_ctju_fin_ult1_diff,...,ind_hip_fin_ult1_diff,ind_plan_fin_ult1_diff,ind_pres_fin_ult1_diff,ind_reca_fin_ult1_diff,ind_tjcr_fin_ult1_diff,ind_valo_fin_ult1_diff,ind_viv_fin_ult1_diff,ind_nomina_ult1_diff,ind_nom_pens_ult1_diff,ind_recibo_ult1_diff
1,2016-05-28,15889,001000001000000000110000,001000001000000000010000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
139,2016-05-28,15988,001000000000000000100000,001000000000000000000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
433,2016-05-28,16525,001010001000110100100111,001010000000110100100111,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
589,2016-05-28,16680,000010000000000011000111,000000000000000011000111,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
655,2016-05-28,16826,001000000000000000001001,001000000000000000001000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
811,2016-05-28,17151,000010010000001001110111,000010010000001001010111,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
877,2016-05-28,17231,001000000000000001110001,001000000000000001010001,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1039,2016-05-28,17458,001000000000000000010000,000000000000000000010000,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1111,2016-05-28,17525,000010011001010000100110,000010011001010000100001,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
1123,2016-05-28,17528,000010000001100001110111,000010000001100001110011,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [16]:
print clients_last_choice.shape
clients_last_choice.head(10)

(149983, 27)


,ncodpers,fecha_dato,targets_str,lc_ind_ahor_fin_ult1,lc_ind_aval_fin_ult1,lc_ind_cco_fin_ult1,lc_ind_cder_fin_ult1,lc_ind_cno_fin_ult1,lc_ind_ctju_fin_ult1,lc_ind_ctma_fin_ult1,...,lc_ind_hip_fin_ult1,lc_ind_plan_fin_ult1,lc_ind_pres_fin_ult1,lc_ind_reca_fin_ult1,lc_ind_tjcr_fin_ult1,lc_ind_valo_fin_ult1,lc_ind_viv_fin_ult1,lc_ind_nomina_ult1,lc_ind_nom_pens_ult1,lc_ind_recibo_ult1
5,15889,2016-05-28,001000001000000000110000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
11,15893,2016-05-28,000000000000000000010000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
17,15895,2016-05-28,001000000000100101110001,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0
23,15897,2016-05-28,000010010000110101110011,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0
29,15900,2016-05-28,000000010000000001000001,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
35,15916,2016-05-28,000010001000100101110001,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0
41,15919,2016-05-28,001000000001000001000001,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
47,15920,2016-05-28,001000010000000001010001,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
53,15921,2016-05-28,001000010001010100110001,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
59,15922,2016-05-28,001000010000000000001001,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


## Another train/predict + CV implementation

### Input

- `X` : `[nb_samples, nb_features]` shaped pd.DataFrame
    - `features_masks_list` : `{fm1_name: features_mask_1, fm2_name: features_mask_2, ...]` with `features_mask_i` is a list of feature column names. They can oversect.
    
- `Y` : `[nb_samples, nb_labels]` shaped pd.DataFrame
    - `labels_masks_list` : `{lm1_name: labels_mask_1, lm2_name: labels_mask_2, ...}` with `labels_mask_i` is a list of labels column names. They can oversect.

- `samples_masks_list` : `[samples_mask_1, samples_mask_2, ...]` with samples_mask_i is a function to produce a boolean pd.DataFrame . Used only for training. 


- Set of models `models` : list of functions to create a model, e.g. `[create_RF, create_NN, create_GBT]`


### Training phase




In [17]:
from collections import defaultdict
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier

In [390]:
samples_masks_list = [
#     'all',
#     lambda x:  ~(x['targets_diff'].isin([0])), 
#     lambda x, y:  x['targets_diff'] > 0, 
#     lambda x, y:  x['targets_diff'] < 0, 
#     lambda x, y:  ~(x['targets_diff'].isin([0])) | ~(y['targets_diff'].isin([0])), 
#     lambda x, y:  ~(y['targets_diff'].isin([0])), 
#     lambda x, y:  (x['targets_diff'] > 0) | (y['targets_diff'] > 0), 
#     lambda x, y:  (x['targets_diff'] < 0) | (y['targets_diff'] < 0), 
    lambda x, y:  (y['targets_diff'] > 0), 
#     lambda x, y:  y['targets_diff'] < 0, 
]

features_masks_dict = {
#     'fm_all': None,
    'fm0': features + target_features + TARGET_LABELS_FRQ.tolist() + TARGET_LABELS_FRQ_PREV,
    'fm1': ['pais_residencia', 'sexo', 'age', 'ind_nuevo', 'segmento', 'ind_empleado', 'ind_actividad_cliente', 'indresi'],
#     'fm2': target_features,
#     'fm3': ['pais_residencia', 'sexo', 'age', 'segmento', 'renta'],
#     'fm4': ['pais_residencia', 'sexo', 'age', 'renta', 'targets_logdiff', 'targets_logcount2_diff','targets_logcount2','targets_logcount1'],
#     'fm5': ['nomprov', 'ind_nuevo', 'renta', 'ind_actividad_cliente', 'canal_entrada'],
#     'fm6': TARGET_LABELS_FRQ,
#     'fm7': TARGET_LABELS_FRQ_PREV,
}

In [391]:
# https://www.analyticsvidhya.com/blog/2016/02/complete-guide-parameter-tuning-gradient-boosting-gbm-python/

def create_RF(input_shape, output_shape):        
    # https://www.analyticsvidhya.com/blog/2015/06/tuning-random-forest-model/
    return RandomForestClassifier(n_estimators=100, 
#                                   min_samples_split=100,
#                                   min_samples_leaf=25,
#                                   max_depth=10
                                  max_features=1.0, 
#                                   oob_score=True,
#                                   bootstrap=True,
                                  n_jobs=-1
                                 )

def create_ET(input_shape, output_shape):
    return ExtraTreesClassifier(n_estimators=100,
#                                   min_samples_leaf=25,
#                                   max_depth=10
                                  max_features=1.0, 
                                  oob_score=True,
                                  bootstrap=True,
                                  n_jobs=-1

                               )

def create_GB(input_shape, output_shape):
    return GradientBoostingClassifier(n_estimators=75)

models_dict = {
    'rf': create_RF,
#     'et': create_ET,
#     'gb': create_GB,
}

In [392]:
NP_TARGET_LABELS = np.array(TARGET_LABELS)
target_labels = TARGET_LABELS_DIFF

common_groups = [
    [2, 3, 4, 5, 16],
    [2, 6, 7, 8, 11],
    [2, 8, 18, 23, 12], 
    [7, 12, 18, 17],
    [12, 11, 23, 5],
#     [2, 18, 23],
#     [18, 23, 21, 22],
    [21, 23, 22, 4],
    [22, 7, 8, 17],
#     [0, 1, 14, 15, 17]
]



def flatten(array):
    out = []
    for item in array:
        out += item
    return out

others = list(set(range(24)) - set(flatten(common_groups)))

# for i, a in enumerate(zip(TARGET_LABELS2, TARGET_LABELS)):
#     print i, a
    
s = set({})
labels_masks_dict = {}
for i, g in enumerate(common_groups):
    print 'lm_%i' % i, " <=> ", g, "<==>", TARGET_LABELS2[g]
    labels_masks_dict['lm_%i' % i] = target_labels[g]
    s |= set(g)
    
print 'lm_others', "<=>", others, "<==>", TARGET_LABELS2[others]
labels_masks_dict['lm_others'] = target_labels[others]
s |= set(others)

assert len(s) == len(target_labels), "Sum is not equal 24, s=%i" % s
# print labels_masks_dict

lm_0  <=>  [2, 3, 4, 5, 16] <==> ['Current Accounts' 'Derivada Account' 'Payroll Account' 'Junior Account'
 'Loans']
lm_1  <=>  [2, 6, 7, 8, 11] <==> ['Current Accounts' 'Mas particular Account' 'particular Account'
 'particular Plus Account' 'Long-term deposits']
lm_2  <=>  [2, 8, 18, 23, 12] <==> ['Current Accounts' 'particular Plus Account' 'Credit Card' 'Direct Debit'
 'e-account']
lm_3  <=>  [7, 12, 18, 17] <==> ['particular Account' 'e-account' 'Credit Card' 'Taxes']
lm_4  <=>  [12, 11, 23, 5] <==> ['e-account' 'Long-term deposits' 'Direct Debit' 'Junior Account']
lm_5  <=>  [21, 23, 22, 4] <==> ['Payroll' 'Direct Debit' 'Pensions' 'Payroll Account']
lm_6  <=>  [22, 7, 8, 17] <==> ['Pensions' 'particular Account' 'particular Plus Account' 'Taxes']
lm_others <=> [0, 1, 9, 10, 13, 14, 15, 19, 20] <==> ['Saving Account' 'Guarantees' 'Short-term deposits' 'Medium-term deposits'
 'Funds' 'Mortgage' 'Pensions (plan fin)' 'Securities' 'Home Account']


In [453]:
# {model_name: [(samples_mask_code, features_mask_name, labels_mask_name), ...]}
models_pipelines = {
    #'gb' : [(None, None, key) for key in labels_masks_dict if len(labels_masks_dict[key]) > 1],
    #'rf' : [(None, None, key) for key in labels_masks_dict if len(labels_masks_dict[key]) > 1],
    #'et' : [(None, None, key) for key in labels_masks_dict if len(labels_masks_dict[key]) > 1],

#    'et': [(0, None, 'lm_1')],
#    'rf': [(0, None, 'lm_1')],
}
models_pipelines

{}

In [454]:
from trainval import train_all, predict_all, probas_to_indices, score_estimators
from utils import map7_score0

In [455]:
ll = 140000
# ll = 1100

mask = (X.index.isin((X.index[:ll//3]))) | (X.index.isin((X.index[2*ll//3:])))

X1 = X[mask]
Y1 = Y[mask]
clc1 = clients_last_choice[mask]
print X1.shape, Y1.shape, clc1.shape

mask = X.index.isin(X.index[ll//3:2*ll//3])
X2 = X[mask]
Y2 = Y[mask]
clc2 = clients_last_choice[mask]
print X2.shape, Y2.shape, clc2.shape

(103316, 75) (103316, 53) (103316, 27)
(46667, 75) (46667, 53) (46667, 27)


In [456]:
from utils import dummies_to_decimal
from sklearn.preprocessing import StandardScaler


def prepare_to_fit(X_train, Y_train):    
    x_train = X_train.values
    x_train = StandardScaler().fit_transform(x_train)
    y_train = Y_train.apply(dummies_to_decimal, axis=1)
    y_train = y_train.values    
    return x_train, y_train


def prepare_to_test(X_val, Y_val=None):
    x_val = X_val.values
    x_val = StandardScaler().fit_transform(x_val)
    if Y_val is not None:
        y_val = Y_val.apply(dummies_to_decimal, axis=1)
        y_val = y_val.values 
    else:
        y_val = None
    return x_val, y_val


def probas_to_labels_probas(y_probas, class_indices, labels):
    l = len(labels)
    out = np.zeros((len(y_probas), l))
    i = 0
    for probas in y_probas:
        if np.sum(probas) > 0:
            pr = np.zeros((l,))
            for index, p in zip(class_indices, probas):
                if index == 0:
                    pr -= p*np.ones((l,))
                else:
                    dummies_str = decimal_to_dummies(index, l)
                    pr += p * np.array([float(v) for v in dummies_str])
            out[i, :] = pr    
        i += 1
    return out
    

In [457]:
# #len(labels_masks_dict['lm_1'])
# res = Y1[labels_masks_dict['lm_1']].apply(dummies_to_decimal, axis=1)
# _uniques = res.unique()
# print _uniques
# print decimal_to_dummies(0, len(labels_masks_dict['lm_1']))

In [458]:
# _y_probas = np.zeros((10, len(_uniques)))

# _y_probas[0, 0] = 0.71
# _y_probas[0, 5] = 0.15
# _y_probas[0, 6] = 0.14
# _y_probas[1, 1] = 1.0
# _y_probas[2, 2] = 0.0
# _y_probas[3, 3] = 1.0
# _y_probas[4, 4] = 1.0
# _y_probas[5, 5] = 0.0
# _y_probas[6, 6] = 1.0
# _y_probas[7, 4] = 1.0
# _y_probas[8, 2] = 1.0
# _y_probas[9, 0] = 0.35
# _y_probas[9, 3] = 0.75

# print "Probas : \n", _y_probas

# # print np.argsort(_y_probas, axis=1)[:, ::-1], np.argmax(_y_probas, axis=1)

# # print "Uniques  :", _uniques, len(_uniques)
# print "Class indices: ", _uniques
# print "Max proba class indices: \n", _uniques[np.argsort(_y_probas, axis=1)[:, ::-1]]

# # print _uniques[np.argmax(_y_probas, axis=1)]

In [459]:
# _l = len(labels_masks_dict['lm_1'])
# class_indices_ = _uniques[np.argsort(_y_probas, axis=1)[:, ::-1]]

# i = 0
# for probas in _y_probas:
#     print "----"
#     print np.sum(probas), np.sum(probas) == 0
#     if np.sum(probas) == 0:
#         continue
#     pr = np.zeros((_l,))
#     for index, p in zip(_uniques, probas):
#         if index == 0:
#             pr -= p * np.ones((_l,))
#         else:
#             dummies_str = decimal_to_dummies(index, _l)
#             pr += p * np.array([float(v) for v in dummies_str])
#             print index, dummies_str, p, pr
#     i += 1
# #     if i == 2:
# #         break

# # _y_labels_probas = probas_to_labels_probas(_y_probas, _uniques, labels_masks_dict['lm_0'])
# # print _y_labels_probas.shape
# # for _y in _y_labels_probas:
# #     print _y

In [460]:
# _Y_probas = pd.DataFrame(index=np.arange(10), columns=labels_masks_dict['lm_0'])
# _Y_probas = _Y_probas.fillna(0.0)

# from trainval import merge_probas

# _Y_probas = merge_probas(_Y_probas, _y_labels_probas, labels_masks_dict['lm_0'], mode='sum')


In [461]:
# _Y_probas.head(10)

In [462]:
_kwargs = {
    'samples_masks_list': samples_masks_list, 
    'features_masks_dict': features_masks_dict, 
    'labels_masks_dict': labels_masks_dict, 
    'models_dict': models_dict,
    'labels': target_labels,
    'transform_proba_func': probas_to_indices,
#     'prepare_to_fit_func': prepare_to_fit,
#     'prepare_to_test_func': prepare_to_test,   
#     'probas_to_labels_probas_func': probas_to_labels_probas,
    'threshold': 0.4,
    'n_highest': 7,
    'mode': 'sum',
    'verbose': False,
    'models_pipelines': models_pipelines,
    'return_probas': True
}

In [463]:
estimators = train_all(X1, Y1, **_kwargs)

INFO:root:-- Train all --
INFO:root:-- Process : sample_mask=3293/103316, features_mask=fm0, labels_mask=lm_others
INFO:root:--- Score : model='rf', fit accuracy : 0.999696
INFO:root:-- Process : sample_mask=3293/103316, features_mask=fm0, labels_mask=lm_0
INFO:root:--- Score : model='rf', fit accuracy : 1.000000
INFO:root:-- Process : sample_mask=3293/103316, features_mask=fm0, labels_mask=lm_1
INFO:root:--- Score : model='rf', fit accuracy : 0.999696
INFO:root:-- Process : sample_mask=3293/103316, features_mask=fm0, labels_mask=lm_2
INFO:root:--- Score : model='rf', fit accuracy : 0.999393
INFO:root:-- Process : sample_mask=3293/103316, features_mask=fm0, labels_mask=lm_3
INFO:root:--- Score : model='rf', fit accuracy : 1.000000
INFO:root:-- Process : sample_mask=3293/103316, features_mask=fm0, labels_mask=lm_4
INFO:root:--- Score : model='rf', fit accuracy : 1.000000
INFO:root:-- Process : sample_mask=3293/103316, features_mask=fm0, labels_mask=lm_5
INFO:root:--- Score : model='rf',

In [464]:
accuracies = defaultdict(list)
for e in estimators:
    accuracies[e[0][2]].append(e[2])

mean_accuracy = {}
for key in accuracies:
    accuracy_list = accuracies[key]
    mean_accuracy[key] = sum(accuracy_list)/len(accuracy_list)
    
mean_accuracy

{'rf': 0.84787807470391741}

In [465]:
_ = score_estimators(estimators, X2, Y2, **_kwargs)

INFO:root:-- Score : model=rf, features_mask=fm0, labels_mask=lm_others -> 0.996614309898
INFO:root:-- Score : model=rf, features_mask=fm0, labels_mask=lm_0 -> 0.903743544689
INFO:root:-- Score : model=rf, features_mask=fm0, labels_mask=lm_1 -> 0.899565003107
INFO:root:-- Score : model=rf, features_mask=fm0, labels_mask=lm_2 -> 0.32952621767
INFO:root:-- Score : model=rf, features_mask=fm0, labels_mask=lm_3 -> 0.956928879079
INFO:root:-- Score : model=rf, features_mask=fm0, labels_mask=lm_4 -> 0.444403968543
INFO:root:-- Score : model=rf, features_mask=fm0, labels_mask=lm_5 -> 0.496882165127
INFO:root:-- Score : model=rf, features_mask=fm0, labels_mask=lm_6 -> 0.972207341376
INFO:root:-- Score : model=rf, features_mask=fm1, labels_mask=lm_others -> 0.999657145306
INFO:root:-- Score : model=rf, features_mask=fm1, labels_mask=lm_0 -> 0.991042921122
INFO:root:-- Score : model=rf, features_mask=fm1, labels_mask=lm_1 -> 0.993292905051
INFO:root:-- Score : model=rf, features_mask=fm1, labels

In [466]:
y_preds, Y_probas = predict_all(estimators, X2, **_kwargs)
# _y_probas = predict_all(estimators, X2, **_kwargs)
#print y_preds[:5]

INFO:root:-- Predict all --


In [467]:
print y_preds[:5]
Y_probas.head()

[[] [2] [21] [23] [23]]


,ind_ahor_fin_ult1_diff,ind_aval_fin_ult1_diff,ind_cco_fin_ult1_diff,ind_cder_fin_ult1_diff,ind_cno_fin_ult1_diff,ind_ctju_fin_ult1_diff,ind_ctma_fin_ult1_diff,ind_ctop_fin_ult1_diff,ind_ctpp_fin_ult1_diff,ind_deco_fin_ult1_diff,...,ind_hip_fin_ult1_diff,ind_plan_fin_ult1_diff,ind_pres_fin_ult1_diff,ind_reca_fin_ult1_diff,ind_tjcr_fin_ult1_diff,ind_valo_fin_ult1_diff,ind_viv_fin_ult1_diff,ind_nomina_ult1_diff,ind_nom_pens_ult1_diff,ind_recibo_ult1_diff
279997,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
280003,0.0,0.0,2.999089,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
280009,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
280015,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
280021,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0


In [468]:
# Y_probas_max = Y_probas.max(axis=1)
# mask = Y_probas_max > 0
# Y_probas.loc[mask, :] = Y_probas[mask].div(Y_probas_max[mask], axis=0)

In [469]:
#labels_masks_dict['lm_0'], common_groups[0]

In [470]:
y_val = targets_str_to_indices(Y2[target_labels].values)
# y_val = targets_str_to_indices(Y2[labels].values)
print y_val[:100]
print y_preds[:100]

[[] [] [] [] [] [] [] [] [] [] [] [] [] [] [] [] [] [] [] [] [] [] [] [] []
 [] [] [] [] [] [] [] [] [] [] [] [] [] [] [] [] [] [] [] [] [] [21, 22] []
 [] [] [] [] [] [] [] [] [] [] [] [] [2, 23] [] [] [] [] [] [] [] [] [] []
 [] [] [] [] [] [] [] [] [] [] [] [] [] [] [] [] [] [] [] [] [] [] [] [] []
 [] [] [] []]
[[] [2] [21] [23] [23] [23] [] [2] [18] [2] [23] [] [2, 23] [] [23] [] []
 [18] [] [18] [23] [18] [] [2] [2] [2, 23] [] [2] [] [2] [2] [2] [18] [23]
 [] [] [18] [4, 19, 2] [] [] [23] [] [] [23] [2] [] [22, 21] [23] [] []
 [21, 22] [] [23] [] [18] [] [2, 23] [] [23] [] [18] [] [] [] [18] [23]
 [22, 21] [23] [22, 23, 21] [2] [23] [] [18] [] [23] [] [23] [] [23]
 [2, 23] [] [23] [] [23] [] [] [18, 19] [18] [] [23] [18] [] [] [4] [23]
 [2] [2, 23] [23, 19] [] [2, 23]]


In [471]:
# np.unique(y_preds)

In [472]:
logging.info("- Compute max map7 score")
map7_score(y_val, y_val, clc2[LC_TARGET_LABELS].values)
# map7_score0(y_val, y_val)
logging.info("- Compute map7 score")
map7_score(y_val, y_preds, clc2[LC_TARGET_LABELS].values)
# map7_score0(y_val, y_preds)

INFO:root:- Compute max map7 score
INFO:root:-- Predicted map7 score: 0.0308569224506
INFO:root:- Compute map7 score
INFO:root:-- Predicted map7 score: 0.0191501608322


0.019150160832184543


On columns lm_0=['ind_cco_fin_ult1', 'ind_cder_fin_ult1', 'ind_cno_fin_ult1','ind_ctju_fin_ult1']

- Feature mask: all : 'fm6', 'fm4', 'fm5', 'fm2', 'fm3', 'fm0', 'fm1'
- threshold = 0.0

Model | Map@7 | Max Map@7 | Labels mask | Samples mask
--- | --- | --- | ---
et | 0.007074370518592629 | 0.0075787893947 | lm_0 | all 



- Feature mask: fm0, fm1, fm3, fm4, fm5
- threshold = 0.0

Model | Map@7 | Max Map@7 | Labels mask | Samples mask
--- | --- | --- | ---
rf + et + gb | 0.006920126730031681 | 0.0075787893947 | lm_0 | all
rf | 0.0068805235951309 | 0.0075787893947 | lm_0 | all
et | 0.006936801734200433 | 0.0075787893947 | lm_0 | all 
gb | 0.0068805235951309 | 0.0075787893947 | lm_0 | all


- Feature mask: fm0, fm1

Model | Map@7 | Max Map@7 | Labels mask | Samples mask
--- | --- | --- | ---
rf + et + gb | 0.004627313656828414 | 0.0075787893947 | lm_0 | all
rf | 0.004664832416208104 | 0.0075787893947 | lm_0 | all
et | 0.004952476238119059 | 0.0075787893947 | lm_0 | all 
gb | 0.004489744872436218 | 0.0075787893947 | lm_0 | all

- Features mask: fm0

Model | Map@7 | Max Map@7 | Labels mask | Samples mask
--- | --- | --- | --- | ---
rf + et + gb | 0.0021010505252626313 | 0.0075787893947 | lm_0 | all
rf + et | 0.001950975487743872 | 0.0075787893947 | lm_0 | all
rf | 0.001550775387693847 | 0.0075787893947 | lm_0 | all
gb | 0.0013006503251625813 | 0.0075787893947 | lm_0 | all
et | 0.0017008504252126063 | 0.0075787893947 | lm_0 | all
et |  0.0014007003501750874 | 0.0075787893947 | lm_0 | x>0 or y>0
rf |  0.0008254127063531766 | 0.0075787893947 | lm_0 | .

--------------------------------------

0.021295269099703414 (GB on 'all')

0.021271936353906683 (RF tunning)

0.021668245671284416 (RF tunning)

0.02136609107928888

0.0211362663776694

In [473]:
# print labels_masks_dict[estimators[0][0][1]]
# print estimators[0][1].classes_
# print estimators[0][1].n_classes_
# print estimators[0][1].n_features_
# print estimators[0][1].n_outputs_
# print estimators[0][1].estimators_

In [474]:
from utils import targets_to_labels, targets_indices_to_labels, remove_last_choice

In [475]:
limit = 25
count = 0

not_predicted_predicted = defaultdict(int)
for last_choice, targets, products, proba in zip(clc2[LC_TARGET_LABELS].values, y_val, y_preds, Y_probas.values):
    added_products = remove_last_choice(targets, last_choice)
    predictions = remove_last_choice(products, last_choice)
#     print "---", count, last_choice
#     print targets, '->', added_products
#     print products, '->', predictions
#     if count == 3:
#         break
    ll = len(added_products)
    
    if ll == 0:
        continue
        
    if len(set(added_products) & set(predictions[:ll])) > 0:
#         print "Predicted : ", added_products, predictions
#         print set(added_products) & set(predictions)
        continue

    count += 1
    if count < limit:
        print "--- Count = ", count
        print targets_indices_to_labels(added_products, TARGET_LABELS2)#, targets_indices_to_labels(targets, TARGET_LABELS2)
        print targets_indices_to_labels(predictions, TARGET_LABELS2), targets_indices_to_labels(products, TARGET_LABELS2)#, proba
    
    for p in added_products:
        not_predicted_predicted[TARGET_LABELS2[p]] += 1
    

--- Count =  1
['Current Accounts', 'Direct Debit']
[] ['Credit Card']
--- Count =  2
['Payroll']
['Direct Debit'] ['Direct Debit']
--- Count =  3
['e-account']
['Direct Debit'] ['Direct Debit']
--- Count =  4
['Credit Card']
[] []
--- Count =  5
['Payroll']
['Direct Debit'] ['Direct Debit']
--- Count =  6
['Credit Card']
[] []
--- Count =  7
['Securities']
[] []
--- Count =  8
['Payroll', 'Pensions']
['Direct Debit'] ['Direct Debit']
--- Count =  9
['Direct Debit']
['Securities', 'Direct Debit'] ['Securities', 'Direct Debit']
--- Count =  10
['Payroll', 'Pensions']
['Direct Debit'] ['Direct Debit']
--- Count =  11
['Payroll', 'Pensions']
['Credit Card'] ['Credit Card']
--- Count =  12
['Credit Card']
['Direct Debit'] ['Direct Debit']
--- Count =  13
['Credit Card']
[] []
--- Count =  14
['Credit Card']
['Payroll Account', 'Pensions', 'Payroll', 'Securities'] ['Payroll Account', 'Pensions', 'Direct Debit', 'Payroll', 'Securities']
--- Count =  15
['Payroll Account', 'Payroll', 'Pension

In [476]:
print not_predicted_predicted, y_val.shape[0]

defaultdict(<type 'int'>, {'Long-term deposits': 3, 'Direct Debit': 43, 'particular Account': 5, 'particular Plus Account': 14, 'e-account': 97, 'Payroll': 89, 'Pensions': 96, 'Taxes': 6, 'Payroll Account': 61, 'Securities': 11, 'Home Account': 1, 'Mas particular Account': 6, 'Funds': 3, 'Credit Card': 181, 'Current Accounts': 39, 'Junior Account': 2, 'Pensions (plan fin)': 1}) 46667


In [241]:
#print y_probas[:10, target_groups[0]]
#print Y[np.array(TARGET_LABELS)[target_groups[0]]].head(10)

### Run KFold Cross-validation 

In [242]:
from trainval import cross_val_score0, cross_val_score

In [244]:
# Unitary run
nb_folds = 3
results = cross_val_score((X, Y, clients_last_choice[LC_TARGET_LABELS].values), nb_folds=nb_folds, **_kwargs)

print "Cross-Validation \n %i | %f | %f | %f | %f | %.5f " % (nb_folds, results.min(), results.mean(), np.median(results), results.max(), results.std())


INFO:root:- Cross validation : 
INFO:root:

		-- Fold : 1 / 3

INFO:root:-- Train all --
INFO:root:-- Process : sample_mask=5384/99988, features_mask=fm0, labels_mask=lm_0
INFO:root:--- Score : model='rf', fit accuracy : 1.000000
INFO:root:-- Process : sample_mask=5384/99988, features_mask=fm0, labels_mask=lm_1
INFO:root:--- Score : model='rf', fit accuracy : 1.000000
INFO:root:-- Process : sample_mask=5384/99988, features_mask=fm0, labels_mask=lm_2
INFO:root:--- Score : model='rf', fit accuracy : 0.999814
INFO:root:-- Process : sample_mask=5384/99988, features_mask=fm0, labels_mask=lm_3
INFO:root:--- Score : model='rf', fit accuracy : 1.000000
INFO:root:-- Process : sample_mask=5384/99988, features_mask=fm0, labels_mask=lm_4
INFO:root:--- Score : model='rf', fit accuracy : 0.999814
INFO:root:-- Process : sample_mask=5384/99988, features_mask=fm0, labels_mask=lm_5
INFO:root:--- Score : model='rf', fit accuracy : 0.999814
INFO:root:-- Process : sample_mask=5384/99988, features_mask=fm0,

KeyboardInterrupt: 


lm_0 + lm_1
3 | 0.004194 | 0.004768 | 0.004746 | 0.005365 | 0.00048 



Samples mask : 'all'
Cross-Validation 
 3 | 0.004092 | 0.004696 | 0.004920 | 0.005077 | 0.00043 

Samples mask : `lambda x, y:  ~(x['targets_diff'].isin([0])) | ~(y['targets_diff'].isin([0]))`
Cross-Validation 
 3 | 0.003892 | 0.004593 | 0.004859 | 0.005028 | 0.00050 



In [86]:
labels_masks_dict['lm_1']

array(['ind_cco_fin_ult1', 'ind_ctma_fin_ult1', 'ind_ctop_fin_ult1',
       'ind_ctpp_fin_ult1'], 
      dtype='|S17')

In [87]:
from itertools import combinations
# CV on various combinations :

_samples_masks_list = [
#     'all',
#     lambda x, y:  ~(x['targets_diff'].isin([0])), 
#     lambda x, y:  x['targets_diff'] > 0, 
#     lambda x, y:  x['targets_diff'] < 0, 
    lambda x, y:  ~(x['targets_diff'].isin([0])) | ~(y['targets_diff'].isin([0])), 
#     lambda x, y:  (x['targets_diff'] > 0) | (y['targets_diff'] > 0), 
#     lambda x, y:  (x['targets_diff'] < 0) | (y['targets_diff'] < 0), 
#     lambda x, y:  (y['targets_diff'] > 0), 
#     lambda x, y:  y['targets_diff'] < 0, 
]

_features_masks_dict = {
    'fm0': features + target_features + TARGET_LABELS_FRQ.tolist() + TARGET_LABELS_FRQ_PREV,
    'fm1': ['pais_residencia', 'sexo', 'age', 'ind_nuevo', 'segmento', 'ind_empleado', 'ind_actividad_cliente', 'indresi'],
#     'fm2': target_features,
    'fm3': ['pais_residencia', 'sexo', 'age', 'segmento', 'renta'],
#     'fm4': ['pais_residencia', 'sexo', 'age', 'renta', 'targets_logdiff', 'targets_logcount2_diff','targets_logcount2','targets_logcount1'],
    'fm5': ['nomprov', 'ind_nuevo', 'renta', 'ind_actividad_cliente', 'canal_entrada'],
#     'fm6': TARGET_LABELS_FRQ,
}

_models_dict = {
    'rf': create_RF,
    'et': create_ET,
#     'gb': create_GB,
}

_labels_masks_dict = {
#     'lm_0': labels_masks_dict['lm_0'],
    'lm_1': labels_masks_dict['lm_1'],
}

nb_folds = 5

def BruteForceSearchCV():
    
    def get_models_combinations(items):
        combins = list(combinations(items, 1))
        combins += list(combinations(items, len(items)))
        return combins
    
    def get_combinations(items):
        combins = list(combinations(items, 1))
        for i in range(2, len(items)+1):
            combins += list(combinations(items, i))
        return combins
    
    def get_items(items):
        out = [[items[0],], ]
        for i in items[1:]:
            tmp = list(out[-1])
            tmp.append(i)
            out.append(tmp)        
        return out

    
    _labels_masks_combinations = get_items(sorted(_labels_masks_dict.keys()))
    _features_masks_combinations = get_items(sorted(_features_masks_dict.keys()))
    _models_combinations = get_models_combinations(_models_dict.keys())

    # Very big loop:
    for lm_keys in _labels_masks_combinations:
        __labels_masks_dict = {}
        for lm_key in lm_keys:
            __labels_masks_dict[lm_key] = _labels_masks_dict[lm_key]

        for i, sm in enumerate(_samples_masks_list):
            __samples_masks_list = [sm]
              
            for fm_keys in _features_masks_combinations:
                __features_masks_dict = {}
                for fm_key in fm_keys:
                    __features_masks_dict[fm_key] = _features_masks_dict[fm_key]
                    
                for m_keys in _models_combinations:
                    __models_dict = {}
                    for m_key in m_keys:
                        __models_dict[m_key] = _models_dict[m_key]
                    
                    print "\n\n---------------------------------------------------------------" 
                    print "--- PROCESS : ", __labels_masks_dict.keys(), i, __features_masks_dict.keys(), __models_dict.keys()
                    print "---------------------------------------------------------------\n" 
                    
                    __kwargs = {
                        'samples_masks_list': __samples_masks_list, 
                        'features_masks_dict': __features_masks_dict, 
                        'labels_masks_dict': __labels_masks_dict, 
                        'models_dict': __models_dict,
                        'labels': target_labels,
                        'transform_proba_func': probas_to_indices,
                        'prepare_to_fit_func': prepare_to_fit,
                        'prepare_to_test_func': prepare_to_test,   
                        'probas_to_labels_probas_func': probas_to_labels_probas,
                        'threshold': 0.0,
                        'n_highest': 7,
                        'mode': 'sum',
                        'verbose': False,
                        'return_probas': True
                    }
                    #  DEBUG : results = cross_val_score((X1, Y1, clc1[LC_TARGET_LABELS].values), nb_folds=nb_folds, **__kwargs)
                    results = cross_val_score((X, Y, clients_last_choice[LC_TARGET_LABELS].values), nb_folds=nb_folds, **__kwargs)
                    print "=> CV : ", results

In [88]:
logging.getLogger().setLevel(logging.WARNING)
BruteForceSearchCV()
logging.getLogger().setLevel(logging.INFO)



---------------------------------------------------------------
--- PROCESS :  ['lm_1'] 0 ['fm0'] ['et']
---------------------------------------------------------------

=> CV :  [ 0.00400873  0.0040004   0.002228    0.00153632  0.00202851]


---------------------------------------------------------------
--- PROCESS :  ['lm_1'] 0 ['fm0'] ['rf']
---------------------------------------------------------------

=> CV :  [ 0.00387261  0.00366148  0.00220578  0.00153076  0.00205166]


---------------------------------------------------------------
--- PROCESS :  ['lm_1'] 0 ['fm0'] ['gb']
---------------------------------------------------------------

=> CV :  [ 0.00441526  0.00433469  0.00249747  0.0016141   0.00210723]


---------------------------------------------------------------
--- PROCESS :  ['lm_1'] 0 ['fm0'] ['et', 'rf', 'gb']
---------------------------------------------------------------

=> CV :  [ 0.00435044  0.00421153  0.00238913  0.00157521  0.00205537]



---------------------------------------------------------------
--- PROCESS :  ['lm_0'] 0 ['fm0'] ['et']
---------------------------------------------------------------

=> CV :  [ 0.00530748  0.0037476   0.00342951  0.00471158  0.00654436]


---------------------------------------------------------------
--- PROCESS :  ['lm_0'] 0 ['fm0'] ['rf']
---------------------------------------------------------------

=> CV :  [ 0.00528664  0.00354452  0.00316559  0.004406    0.0059906 ]


---------------------------------------------------------------
--- PROCESS :  ['lm_0'] 0 ['fm0'] ['gb']
---------------------------------------------------------------

=> CV :  [ 0.00453656  0.00385566  0.0035573   0.00474492  0.00669623]


---------------------------------------------------------------
--- PROCESS :  ['lm_0'] 0 ['fm0'] ['et', 'rf', 'gb']
---------------------------------------------------------------

=> CV :  [ 0.00556167  0.00380288  0.00356286  0.00480048  0.00664622]


---------------------------------------------------------------
--- PROCESS :  ['lm_0'] 0 ['fm0', 'fm1'] ['et']
---------------------------------------------------------------

=> CV :  [ 0.0051144   0.00350424  0.00357119  0.00466713  0.00656788]


---------------------------------------------------------------
--- PROCESS :  ['lm_0'] 0 ['fm0', 'fm1'] ['rf']
---------------------------------------------------------------

=> CV :  [ 0.00530609  0.00353341  0.00353785  0.00450045  0.0064512 ]


---------------------------------------------------------------
--- PROCESS :  ['lm_0'] 0 ['fm0', 'fm1'] ['gb']
---------------------------------------------------------------

=> CV :  [ 0.00541721  0.00355563  0.00356563  0.00464769  0.006654  ]


---------------------------------------------------------------
--- PROCESS :  ['lm_0'] 0 ['fm0', 'fm1'] ['et', 'rf', 'gb']
---------------------------------------------------------------

=> CV :  [ 0.00528247  0.00358064  0.00356563  0.00457824  0.00661233]


---------------------------------------------------------------
--- PROCESS :  ['lm_0'] 0 ['fm2', 'fm0', 'fm1'] ['et']
---------------------------------------------------------------

Compute cross-validation across several months

In [ ]:
nb_folds = 3
yms = [201504, 201505]
#yms = [201505]

for ym in yms:
    logging.info("\n-------------------------")
    logging.info("- Process month : %s" % ym)
    logging.info("-------------------------\n")
    
    ym1 = ym + 100    
    df1 = train_df if months_ym_map[ym] in train_months else val_df
    df2 = train_df if months_ym_map[ym1] in train_months else val_df
    X, Y, clients_last_choice = get_XY(ym, df1, ym1, df2) 
    results = cross_val_score2((X, Y, clients_last_choice[LC_TARGET_LABELS].values), 
                                profiles=profiles,
                                nb_folds=nb_folds)
    print "Cross-Validation \n %i | %f | %f | %f | %.5f " % (nb_folds, results.min(), results.mean(), results.max(), results.std())

## Train model for predictions

In [ ]:
current_month = 201505
next_year_month = current_month + 100

df1 = train_df
#df1 = val_df
df2 = train_df #if months_ym_map[next_year_month] in train_months else val_df
#df2 = val_df

X, Y, clients_last_choice = get_XY(current_month, df1, next_year_month, df2, months_ym_map)

In [ ]:
estimators = train_all(X, Y, **_kwargs)

In [ ]:
y_preds, Y_probas = predict_all(estimators, X, **_kwargs)

Check score on the data 2016/05

In [ ]:
logging.info("- Compute map7 score")
print map7_score(y_val, y_preds, clients_last_choice[LC_TARGET_LABELS].values)
logging.info("- Compute max map7 score")
print map7_score(y_val, y_val, clients_last_choice[LC_TARGET_LABELS].values)

## Prediction for 2016/06

In [ ]:
from dataset import load_train_test

In [ ]:
full_train_df, test_df = load_train_test([201506])

In [ ]:
full_train_df.head()

In [ ]:
test_df.head()

In [ ]:
months_ym_map = {}
months = list(set(full_train_df['fecha_dato'].unique()) | set(test_df['fecha_dato'].unique()))
for m in months:
    months_ym_map[to_yearmonth(m)] = m
    
full_train_months = full_train_df['fecha_dato'].unique()
test_months = test_df['fecha_dato'].unique()

In [ ]:
current_month = 201506
next_year_month = current_month + 100

df1 = full_train_df
df2 = test_df
X, _, clients_last_choice = get_XY(current_month, df1, next_year_month, df2, months_ym_map)

In [ ]:
print X.shape, test_df.shape

In [ ]:
X.head(10)

In [ ]:
clients_last_choice.head(10)

In [ ]:
def get_submission(predicted_added_products, clients, clc, target_labels):
    added_products_col = []
    count = 0 
    for products, last_choice in zip(predicted_added_products, clc):
        predictions = remove_last_choice(products, last_choice)
        added_products_col.append(' '.join([target_labels[i] for i in predictions]))
        count+=1
        if count % 100000 == 0:
            logging.info("Elapsed : %i", count)
            
    out = pd.DataFrame(data={'ncodpers': clients, 'added_products': added_products_col}, columns=['ncodpers', 'added_products'])
    return out

In [ ]:
y_preds, Y_probas = predict_all(estimators, X, **_kwargs)

logging.info("- Get submission dataframe:")
clients = X['ncodpers'].values
submission = get_submission(y_pred, clients, clients_last_choice[TARGET_LABELS].values, TARGET_LABELS)

In [ ]:
selected_estimators = []
for e in estimators:
    # estimators = [([features_mask_name, labels_mask_name, model_name], estimator_object, accuracy), ...]
    features_mask_name, labels_mask_name, model_name = e[0]
#     if features_masks_dict[features_mask_name]
#     if e[0]

In [ ]:
features_masks_dict[features_mask_name] in test_df.columns

In [ ]:
submission_clients = set(submission['ncodpers'].unique())
test_clients = set(test_df['ncodpers'].unique())
if submission_clients != test_clients:
    missing_clients = list(test_clients - submission_clients)
    missing_clients_mask = test_df['ncodpers'].isin(missing_clients)
    
    X1 = test_df[missing_clients_mask]
    
    

        
    
    missing_added_products = np.zeros((len(missing_clients)))
    submission = pd.concat([submission, 
                            pd.DataFrame(data={
                                'ncodpers': missing_clients, 
                                'added_products': missing_added_products
                            }, columns=['ncodpers', 'added_products'])])

Get submission DataFrame and write csv file

In [ ]:
print submission.shape
submission.head()

In [ ]:
from datetime import datetime
import csv

logging.info('- Generate submission')
submission_file = '../results/submission_' + \
                  str(datetime.now().strftime("%Y-%m-%d-%H-%M")) + \
                  '.csv'

submission.to_csv(submission_file, index=False, index_label=False)

In [ ]:
with open('../results/submission_2016-11-17-16-37.csv', 'r') as r:
    print r.readline()
    print r.readline()
    print r.readline()
    print r.readline()
    print r.readline()
    